In [36]:
import pandas as pd
import numpy as np

from googleapiclient.discovery import build
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors


from dateutil import parser
import pandas as pd
from IPython.display import JSON

import os
import requests
import pickle

from dotenv import load_dotenv 
load_dotenv()


True

In [21]:
api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = os.getenv('yt_key')

youtube = googleapiclient.discovery.build(
api_service_name, api_version, developerKey = DEVELOPER_KEY)

In [51]:
def get_travel_videos_ids(youtube, search):

    request = youtube.search().list(
    
    part="id,snippet",
    maxResults=50,
    order="relevance",
    q=search,
    relevanceLanguage="en",
    type="video",
    videoCategoryId="19"
    )

    response = request.execute()

    all_videos = []

    for i in range(len(response['items'])):

        all_videos.append(response['items'][i]['id']['videoId'])

    next_page_token = response.get('nextPageToken')
    more_pages = True

    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.search().list(

            part="id,snippet",
            maxResults=50,
            order="relevance",
            q=search,
            relevanceLanguage="en",
            type="video",
            videoCategoryId="19",
            pageToken = next_page_token
            )

            response = request.execute()

            for i in range(len(response['items'])):
                all_videos.append(response['items'][i]['id']['videoId'])

            next_page_token = response.get('nextPageToken')
                
    return all_videos


In [38]:
def get_video_details(youtube, video_ids):
    
    """
    Get video statistics of all videos with given IDs
    Params:
    
    youtube: the build object from googleapiclient.discovery
    video_ids: list of video IDs
    
    Returns:
    Dataframe with statistics of videos, i.e.:
        'channelTitle', 'title', 'description', 'tags', 'publishedAt'
        'viewCount', 'likeCount', 'favoriteCount', 'commentCount'
        'duration', 'caption'(boolean)
    """
        
    all_video_info = []
    
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute() 

        for video in response['items']:
            stats_to_keep = {
                             'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                             'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                             'contentDetails': ['duration', 'caption']
                            }
            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None

            all_video_info.append(video_info)
            
    return pd.DataFrame(all_video_info)


In [33]:
#all_videos = get_travel_videos_ids(youtube, "travel|trip|vacation")

In [43]:
len(all_videos)

599

In [45]:
#all_videos2 = get_travel_videos_ids(youtube, "best place to visit")

In [46]:
len(all_videos2)

600

In [37]:
# to save:
# with open('all_videos.txt', 'wb') as fp:
#     pickle.dump(all_videos, fp)
    
# to read:
# with open ('/content/list_1.ob', 'rb') as fp:
#     list_1 = pickle.load(fp)

In [39]:
videos_details = get_video_details(youtube, all_videos)

In [40]:
videos_details

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,caption
0,wLSyTqAzUMc,Backpacking Bananas,Copacabana & Isla Del Sol 🇧🇴 Backpacking Boliv...,My final stop in Bolivia is Copacabana! Situat...,"[gap year ideas, gap year travel, travel tips,...",2022-11-22T15:15:04Z,8058,498,None,38,PT13M37S,false
1,UdpRM5zEoBw,BEYOND THE TRIP - Viaggio a Tempo Indeterminato,E ALLA FINE ARRIVA LA FREGATURA 🌏 Prima o poi ...,🚙Ep. 312 - Arriviamo a Dwarka senza aspettativ...,"[beyond the trip, angela e paolo viaggio, ange...",2022-11-23T12:00:09Z,11078,1647,None,80,PT25M30S,false
2,w9bfgLz9v6Q,DW Travel,What Tourists Love To Do in Berlin | Berlin To...,What to do in Berlin? Fine dining or doner keb...,"[things to do in berlin, visit berlin, berlin ...",2022-11-23T14:00:36Z,2155,122,None,10,PT8M7S,true
3,ruqnVKUquiE,Justin Tse,ITALY // My Ferrari Road Trip Experience in Fl...,Beyond by Four Seasons - https://www.fourseaso...,"[justin tse, Italy, Four seasons, Four seasons...",2022-11-22T21:54:10Z,11199,662,None,28,PT13M55S,false
4,VZIAOBl49_8,Travel Trolls TV,We Found Paradise In SPAIN for Just £15!,Our first walk on the beach in beautiful Spain...,"[motorhome spain, motor homing spain, winter i...",2022-11-23T17:30:03Z,6554,654,None,113,PT48M54S,false
...,...,...,...,...,...,...,...,...,...,...,...,...
594,Xf53mfnlGGg,Breakaway On A Budget,This Travel Tip Will Save You A Lot Of Money o...,Tip to help you save a little money before vac...,"[Travel tips, air travel, airport hacks, flyin...",2022-11-23T11:47:18Z,136,3,None,0,PT13S,false
595,8ZLfHfFGCwI,Travel with Amit,How to Plan Budget Trip to Andaman | All about...,This video is the first part of my Andaman tou...,"[travel with amit, andaman tour guide, how to ...",2018-07-07T09:12:09Z,476456,9626,None,714,PT10M59S,false
596,I_PmMn8MO8M,magroswelt,Aruba the best of vacation 4K,"Aruba is not only a great cruise destination, ...","[Aruba, Best of, vacation, 4K, cruise, destina...",2018-09-19T18:04:31Z,46196,121,None,15,PT3M38S,false
597,DHqJc059VBM,Rick Steves' Europe,"Orkney, Scotland: Scapa Flow and WWII - Rick S...",Watch the full episode: https://youtu.be/CsR2X...,"[Rick Steves, Rick Steves Europe, Europe trave...",2022-11-23T08:00:12Z,4350,359,None,19,PT2M38S,true


In [42]:
#videos_details.to_csv('video_details.csv')

In [47]:
videos_details2 = get_video_details(youtube, all_videos2)

In [48]:
videos_details2

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,caption
0,L5JORXmV_A0,touropia,10 Best Places to Visit in Spain - Travel Video,Check out all the places seen in this video: h...,"[spain, spain travel, spain tourism, barcelona...",2020-07-03T13:24:52Z,1850031,17287,None,1007,PT16M41S,true
1,zTjnQ0XlAc4,touropia,10 Best Places to Visit in France - Travel Video,Check out all the places seen in this video: h...,"[Marseille, Lyon, Paris, Cannes, Nice, Dordogn...",2018-09-21T11:43:39Z,1258382,17330,None,1025,PT12M48S,true
2,0GZSfBuhf6Y,touropia,25 Best Places to Visit in Europe - Travel Europe,Check out all the places seen in this video: h...,"[europe travel, best places to visit in europe...",2019-06-11T13:56:59Z,9563730,74744,None,3791,PT25M55S,true
3,IgAnj6r1O48,touropia,10 Best Places to Visit in India - Travel Video,Check out all the places seen in this video: h...,"[india places, best places to visit in india, ...",2019-05-09T10:06:27Z,5353223,54500,None,5317,PT13M30S,true
4,kmJt1wViRXA,touropia,10 Best Places to Visit in North Carolina - Tr...,"A state of many contrasts, North Carolina is h...","[north carolina, north carolina tourism, north...",2022-11-18T11:56:26Z,45930,639,None,65,PT12M55S,false
...,...,...,...,...,...,...,...,...,...,...,...,...
595,hpL65C-n8xA,Yaduvansi 23,Top 7 Best Place to visit in uttar pradesh / उ...,Top 7 Best Place to visit in uttar pradesh / उ...,None,2022-11-22T15:15:11Z,1,0,None,0,PT4M52S,false
596,y7erADpGmTI,Kinetic Kennons,"First Impressions of Doha, Qatar (World Cup 2022)","Welcome to Doha, Qatar- home of the 2022 World...","[Kinetic Kennons, travel, bucket list, kinetic...",2022-11-19T14:00:29Z,126856,2323,None,498,PT13M41S,false
597,t2-yqy0N7b0,Let's Move Travel & Tour . 489k views . 2 hour...,kumrat Kala Chashma | Kumrat Valley | Best Pla...,#kumratvalley #tourisminpakistan #kalachashma ...,None,2022-11-22T14:14:30Z,13,6,None,1,PT2M23S,false
598,Rn-UBRSu5TU,goandeuropetravel,3 BEST PLACES TO TRAVEL ALONE IN EUROPE | Wher...,3 BEST PLACES TO TRAVEL ALONE IN EUROPE | Wher...,"[travel guide, travel tips, tourist attracatio...",2022-11-18T17:00:22Z,62,2,None,0,PT16S,false


In [49]:
#videos_details2.to_csv('video_details_bestplaces.csv')